# Imports 

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Setup

In [2]:
spark = (
    SparkSession.builder.appName("pyspark-notebook")
    .master("spark://spark-master:7077")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f4627c01-039e-4e69-a167-16368d67769e;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 288ms :: artifacts dl 13ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	io.delta#delta-core_2.12;2.1.0 from central in [default]
	io.delta#delta-storage;2.1.0 

23/04/12 07:41:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", "minio")
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "minio123")
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://172.18.0.2:9000")

In [4]:
from delta.tables import *

# Create initial dataframe

In [5]:
df = (
    spark
    .read
    .option("header", "true")
    .csv("s3a://incoming/")
)

23/04/12 07:41:44 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


23/04/12 07:42:06 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:42:21 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:42:36 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:42:51 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


23/04/12 07:43:06 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:43:21 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:43:36 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:43:51 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


23/04/12 07:44:06 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:44:21 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:44:36 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:44:51 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


23/04/12 07:45:06 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:45:21 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:45:36 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:45:51 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


23/04/12 07:46:06 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:46:21 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:46:36 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:46:51 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


23/04/12 07:47:06 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:47:21 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:47:36 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:47:51 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


23/04/12 07:48:06 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:48:21 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:48:36 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:48:51 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


23/04/12 07:49:06 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:49:21 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:49:36 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/12 07:49:51 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


23/04/12 07:50:06 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [6]:
df = df.withColumn("source_filename", F.input_file_name())

In [13]:
df.show(1, truncate=False, vertical=True)

-RECORD 0----------------------------------------------------------------
 vendor_id           | 1                                                 
 pickup_datetime     | 2019-01-15 03:36:12                               
 dropoff_datetime    | 2019-01-15 03:42:19                               
 passenger_count     | 1                                                 
 pickup_location_id  | 230                                               
 dropoff_location_id | 48                                                
 fare_amount         | 6.5                                               
 source_filename     | s3a://incoming/yellow_tripdata_sample_2019_01.csv 
only showing top 1 row



In [15]:
(
    df
    .write
    .format("delta")
    .save("s3a://lake/taxis")
)

# Check schemas

In [9]:
dl = DeltaTable.forPath(spark, "s3a://lake/taxis")

In [10]:
dl.history().show(truncate=False, vertical=True)

-RECORD 0---------------------------------------------------------------------------
 version             | 0                                                            
 timestamp           | 2023-04-12 07:50:29                                          
 userId              | null                                                         
 userName            | null                                                         
 operation           | WRITE                                                        
 operationParameters | {mode -> ErrorIfExists, partitionBy -> []}                   
 job                 | null                                                         
 notebook            | null                                                         
 clusterId           | null                                                         
 readVersion         | null                                                         
 isolationLevel      | Serializable                              

# Check data 

In [11]:
df = spark.read.format("delta").load("s3a://lake/taxis")

In [12]:
df.show()

+---------+-------------------+-------------------+---------------+------------------+-------------------+-----------+--------------------+
|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|pickup_location_id|dropoff_location_id|fare_amount|     source_filename|
+---------+-------------------+-------------------+---------------+------------------+-------------------+-----------+--------------------+
|        1|2019-01-15 03:36:12|2019-01-15 03:42:19|              1|               230|                 48|        6.5|s3a://incoming/ye...|
|        1|2019-01-25 18:20:32|2019-01-25 18:26:55|              1|               112|                112|        6.0|s3a://incoming/ye...|
|        1|2019-01-05 06:47:31|2019-01-05 06:52:19|              1|               107|                  4|        6.0|s3a://incoming/ye...|
|        1|2019-01-09 15:08:02|2019-01-09 15:20:17|              1|               143|                158|       11.0|s3a://incoming/ye...|
|        1|2019-01-2